In [1]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/mounts/work/faeze/.cache/hf/'
os.environ['HF_HOME'] = '/mounts/work/faeze/.cache/hf/'
os.environ['HF_DATASETS_CACHE'] = '/mounts/work/faeze/.cache/hf/'
os.environ['TORCH_HUB'] = '/mounts/work/faeze/.cache/torch/'
os.environ['TORCH_HOME'] = '/mounts/work/faeze/.cache/torch/'
os.environ["WANDB_DIR"] = '/mounts/work/faeze/.cache/wandb/'

In [2]:
from data_provider import DataProvider
from finetuner import FineTuner, FineTunerConfig

/mounts/Users/cisintern/faeze/.local/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [3]:
data_provider_ = DataProvider()

In [4]:
names = ['bas19_es', 'dyn21_en', 'for19_pt', 'fou18_en', 'has21_hi', 'ken20_en',
                       'ous19_ar', 'ous19_fr', 'san20_it',]

datas = [f'baseline_data/{i}' for i in names]
languages = [name.split('_')[1] for name in names]
languages

['es', 'en', 'pt', 'en', 'hi', 'en', 'ar', 'fr', 'it']

In [5]:
datasets = data_provider_.load_datasets(datas, languages)

/mounts/work/faeze/new_miniconda3/envs/2024_9/lib/python3.10/site-packages/datasets/load.py:929: FutureWarning: The repository for baseline_data contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at baseline_data/baseline_data.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [6]:
datasets[5]

{'name': 'baseline_data/ken20_en',
 'split': 'test',
 'data': DatasetDict({
     train: Dataset({
         features: ['id', 'text', 'label'],
         num_rows: 20692
     })
     validation: Dataset({
         features: ['id', 'text', 'label'],
         num_rows: 500
     })
     test: Dataset({
         features: ['id', 'text', 'label'],
         num_rows: 2000
     })
 }),
 'language': 'en'}

In [14]:
train_data = datasets[0]['data']['train']
eval_data = datasets[0]['data']['validation']
test_data = datasets[0]['data']['test']

In [15]:
datasets

[{'name': 'baseline_data/bas19_es',
  'split': 'test',
  'data': DatasetDict({
      train: Dataset({
          features: ['id', 'text', 'label'],
          num_rows: 4100
      })
      validation: Dataset({
          features: ['id', 'text', 'label'],
          num_rows: 500
      })
      test: Dataset({
          features: ['id', 'text', 'label'],
          num_rows: 2000
      })
  }),
  'language': 'es'},
 {'name': 'baseline_data/dyn21_en',
  'split': 'test',
  'data': DatasetDict({
      train: Dataset({
          features: ['id', 'text', 'label'],
          num_rows: 38644
      })
      validation: Dataset({
          features: ['id', 'text', 'label'],
          num_rows: 500
      })
      test: Dataset({
          features: ['id', 'text', 'label'],
          num_rows: 2000
      })
  }),
  'language': 'en'},
 {'name': 'baseline_data/for19_pt',
  'split': 'test',
  'data': DatasetDict({
      train: Dataset({
          features: ['id', 'text', 'label'],
          num_rows: 31

In [13]:


config = FineTunerConfig(
        model_name="FacebookAI/xlm-roberta-base",
        fine_tune_method="default",
        num_labels=2,  # Binary classification (Hate Speech detection)
        learning_rate=5e-5,
        epochs=5,
        batch_size=16,
    # peft_config={"lora_rank": 4}
    )

In [16]:
fine_tuner = FineTuner(config)

/mounts/Users/cisintern/faeze/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
train_dataset = fine_tuner.prepare_data(train_data)
eval_dataset = fine_tuner.prepare_data(eval_data)
test_dataset = fine_tuner.prepare_data(test_data)

In [18]:
fine_tuner.train(train_dataset, eval_dataset)

/mounts/Users/cisintern/faeze/.local/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/mounts/Users/cisintern/faeze/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,F1-macro,Precision,Recall,F1-weighted
1,0.669700,0.565356,0.698000,0.671097,0.691927,0.698000,0.691792
2,0.519600,0.405063,0.834000,0.817341,0.837512,0.834000,0.829477
3,0.434600,0.387590,0.830000,0.815761,0.830115,0.830000,0.827029
4,0.329800,0.426065,0.826000,0.821995,0.837824,0.826000,0.827869
5,0.253600,0.384766,0.852000,0.844757,0.852296,0.852000,0.852134


/mounts/Users/cisintern/faeze/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/mounts/Users/cisintern/faeze/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/mounts/Users/cisintern/faeze/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/mounts/Users/cisintern/faeze/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68

In [12]:
prediction = fine_tuner.predict(test_dataset)
results = fine_tuner.compute_metrics(prediction)
print(results)

/mounts/Users/cisintern/faeze/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'accuracy': 0.8015, 'f1-macro': 0.7990011318210826, 'precision': 0.8074411137805951, 'recall': 0.8015, 'f1-weighted': 0.8025869516293944}


In [13]:
{'accuracy': 0.782, 'f1-macro': 0.7790265816831281, 'precision': 0.7872806227483272, 'recall': 0.782, 'f1-weighted': 0.7831278483270894}

{'accuracy': 0.782,
 'f1-macro': 0.7790265816831281,
 'precision': 0.7872806227483272,
 'recall': 0.782,
 'f1-weighted': 0.7831278483270894}

In [14]:
{'accuracy': 0.758, 'f1-macro': 0.715371469635899, 'precision': 0.7543940303940304, 'recall': 0.758, 'f1-weighted': 0.7559071264162328}

{'accuracy': 0.758,
 'f1-macro': 0.715371469635899,
 'precision': 0.7543940303940304,
 'recall': 0.758,
 'f1-weighted': 0.7559071264162328}